#### **Importing Packages**

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#### **Load the data**

In [2]:
df_preprocessed = pd.read_csv('processed_data.csv')

In [3]:
df_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month of the Year,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,0


In [4]:
df_preprocessed.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4',
       'Month of the Year', 'Day of the Week', 'Transportation Expense',
       'Distance to Work', 'Age', 'Daily Work Load Average',
       'Body Mass Index', 'Education', 'Children', 'Pets',
       'Excessive Absenteeism'], dtype=object)

In [5]:
df_preprocessed = df_preprocessed.drop(['Education','Distance to Work', 'Day of the Week'], axis = 1)

In [6]:
target_one = df_preprocessed['Excessive Absenteeism'].sum()
lenght_ea = len(df_preprocessed['Excessive Absenteeism'])
print('How balanced is target: ', target_one/lenght_ea)

How balanced is target:  0.45571428571428574


In [7]:
df_preprocessed.shape

(700, 12)

#### **Select inputs for the regression**

In [8]:
unscaled_inputs = df_preprocessed.iloc[:,:-1]
target = df_preprocessed.iloc[:,-1]

#### **Standardize the data**

In [9]:
exclude_cols = [
    'Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Education'
]

In [10]:
def scale_features(unscaled_inputs, exclude_cols):
    cols_to_scale = [col for col in unscaled_inputs.columns if col not in exclude_cols]
    absenteeism_scaler = StandardScaler()
    scaled_inputs = unscaled_inputs.copy()
    scaled_inputs[cols_to_scale] = absenteeism_scaler.fit_transform(unscaled_inputs[cols_to_scale])
    return scaled_inputs, absenteeism_scaler

In [11]:
scaled_inputs, absenteeism_scaler = scale_features(unscaled_inputs, exclude_cols)

In [12]:
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month of the Year,Transportation Expense,Age,Daily Work Load Average,Body Mass Index,Children,Pets
0,0,0,0,1,0.182726,1.005844,-0.536062,-0.806331,0.767431,0.880469,0.268487
1,0,0,0,0,0.182726,-1.574681,2.130803,-0.806331,1.002633,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.654143,0.248310,-0.806331,1.002633,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.854936,0.405184,-0.806331,-0.643782,0.880469,-0.589690
4,0,0,0,1,0.182726,1.005844,-0.536062,-0.806331,0.767431,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.654143,0.562059,-0.853789,-1.114186,0.880469,-0.589690
696,1,0,0,0,-0.388293,0.040034,-1.320435,-0.853789,-0.643782,-0.019280,1.126663
697,1,0,0,0,-0.388293,1.624567,-1.320435,-0.853789,-0.408580,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.190942,-0.692937,-0.853789,-0.408580,-0.919030,-0.589690


#### **Splitting the Data**

In [13]:
X_train, X_test, y_train, y_test = train_test_split(scaled_inputs, target, train_size = .8, random_state = 365)

In [14]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(560, 11) (560,)
(140, 11) (140,)


#### **Model Building & Prediction**

In [15]:
LogReg = LogisticRegression()
history = LogReg.fit(X_train, y_train)

In [16]:
LogReg.score(X_train, y_train)

0.7714285714285715

In [17]:
model_output = LogReg.predict(X_train)

##### **Manual Accuracy Score Calculation**

In [18]:
# Manual Calculation
print('Predicted = Actual: ', np.sum(model_output == y_train))
print('Total Predictions: ', model_output.shape[0])
Acc = (np.sum(model_output == y_train))/model_output.shape[0]
print('Accuracy Manually Calculated: ', Acc)

Predicted = Actual:  432
Total Predictions:  560
Accuracy Manually Calculated:  0.7714285714285715


#### **Coefficients (Weights) & Intercept**

In [19]:
LogReg.coef_

array([[ 2.67898219,  1.31612516,  2.95989593,  0.86639296,  0.14461204,
         0.50391264, -0.24812832, -0.01049201,  0.33900401,  0.4419262 ,
        -0.30237637]])

In [20]:
LogReg.intercept_

array([-1.57940652])

In [21]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4',
       'Month of the Year', 'Transportation Expense', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Children', 'Pets'],
      dtype=object)

In [22]:
features_name = unscaled_inputs.columns.values

In [23]:
summary_table = pd.DataFrame(columns = ['Feature name'], data = features_name)
summary_table['Coefficient'] = np.transpose(LogReg.coef_)

In [24]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', LogReg.intercept_[0]]
summary_table = summary_table.sort_index()

In [25]:
summary_table['Odds Ratio'] = np.exp(summary_table['Coefficient'])

In [26]:
summary_table.sort_values('Odds Ratio', ascending = False)

,Feature name,Coefficient,Odds Ratio
3,Reason_3,2.959896,19.295963
1,Reason_1,2.678982,14.570256
2,Reason_2,1.316125,3.728944
4,Reason_4,0.866393,2.378317
6,Transportation Expense,0.503913,1.655185
10,Children,0.441926,1.555701
9,Body Mass Index,0.339004,1.403549
5,Month of the Year,0.144612,1.155591
8,Daily Work Load Average,-0.010492,0.989563
7,Age,-0.248128,0.780260


#### **Testing the model**

In [27]:
LogReg.score(X_test, y_test)

0.7642857142857142

In [28]:
final_outputs_proba = LogReg.predict_proba(X_test)
final_outputs_proba

array([[0.52265689, 0.47734311],
       [0.57788649, 0.42211351],
       [0.42388507, 0.57611493],
       [0.16499202, 0.83500798],
       [0.20888434, 0.79111566],
       [0.45002504, 0.54997496],
       [0.19118422, 0.80881578],
       [0.56328362, 0.43671638],
       [0.53577301, 0.46422699],
       [0.32384674, 0.67615326],
       [0.84383077, 0.15616923],
       [0.84920203, 0.15079797],
       [0.70232578, 0.29767422],
       [0.41013655, 0.58986345],
       [0.30633006, 0.69366994],
       [0.69077884, 0.30922116],
       [0.50348921, 0.49651079],
       [0.50495259, 0.49504741],
       [0.67724876, 0.32275124],
       [0.76259676, 0.23740324],
       [0.90678356, 0.09321644],
       [0.70583463, 0.29416537],
       [0.31622248, 0.68377752],
       [0.49167754, 0.50832246],
       [0.84627108, 0.15372892],
       [0.48363142, 0.51636858],
       [0.86844753, 0.13155247],
       [0.75383175, 0.24616825],
       [0.36990273, 0.63009727],
       [0.69294855, 0.30705145],
       [0.

In [29]:
final_outputs_proba.shape

(140, 2)

In [30]:
final_outputs_proba[:,1]

array([0.47734311, 0.42211351, 0.57611493, 0.83500798, 0.79111566,
       0.54997496, 0.80881578, 0.43671638, 0.46422699, 0.67615326,
       0.15616923, 0.15079797, 0.29767422, 0.58986345, 0.69366994,
       0.30922116, 0.49651079, 0.49504741, 0.32275124, 0.23740324,
       0.09321644, 0.29416537, 0.68377752, 0.50832246, 0.15372892,
       0.51636858, 0.13155247, 0.24616825, 0.63009727, 0.30705145,
       0.72195734, 0.23677456, 0.5853107 , 0.67362312, 0.39849034,
       0.74096002, 0.64676002, 0.57936404, 0.57750743, 0.7060223 ,
       0.89567375, 0.6731511 , 0.24587469, 0.24587469, 0.25159617,
       0.92426117, 0.63744059, 0.38336856, 0.77108128, 0.30920756,
       0.23740324, 0.44333849, 0.84804702, 0.20280005, 0.48568579,
       0.7016732 , 0.41871751, 0.25329082, 0.23681333, 0.40706559,
       0.30214676, 0.35651397, 0.65386228, 0.52670962, 0.40210757,
       0.58629811, 0.73440013, 0.20280005, 0.23796599, 0.30795857,
       0.6969363 , 0.15400365, 0.09662695, 0.49449423, 0.33135

#### **Save the model**

In [31]:
import pickle

In [32]:
with open('model','wb') as file:
    pickle.dump(LogReg, file)

In [33]:
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)